In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
eval_iters= 200
learning_rate = 3e-4

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[46, 29,  1, 28, 25, 38, 31, 29],
        [54, 67, 57, 59, 54, 73, 61, 58],
        [58,  1, 54,  1, 66, 58, 67, 54],
        [71, 54, 65,  0, 68, 73, 61, 58]])
targets:
tensor([[29,  1, 28, 25, 38, 31, 29, 42],
        [67, 57, 59, 54, 73, 61, 58, 71],
        [ 1, 54,  1, 66, 58, 67, 54, 60],
        [54, 65,  0, 68, 73, 61, 58, 71]])


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("When input is", context, "target is", target)

When input is tensor([80]) target is tensor(1)
When input is tensor([80,  1]) target is tensor(1)
When input is tensor([80,  1,  1]) target is tensor(28)
When input is tensor([80,  1,  1, 28]) target is tensor(39)
When input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
When input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
When input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
When input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :]
            #apply softmax function to get probabilities
            probs = F.softmax(logits, dim=-1)
            #sample from the dristribution
            index_next = torch.multinomial(probs, num_samples=1)
            #append sampled index to the runing sequence
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [7]:
@torch.no_grad()
def estimate_loss():
    out= {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses= estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    
    #sample a batch of data
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits, loss= model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.8953, val loss: 4.8724
step: 200, train loss: 4.8387, val loss: 4.8512
step: 400, train loss: 4.7799, val loss: 4.7715
step: 600, train loss: 4.7589, val loss: 4.7364
step: 800, train loss: 4.6956, val loss: 4.6802
step: 1000, train loss: 4.6243, val loss: 4.6337
step: 1200, train loss: 4.6089, val loss: 4.5860
step: 1400, train loss: 4.5401, val loss: 4.5295
step: 1600, train loss: 4.5030, val loss: 4.5085
step: 1800, train loss: 4.4718, val loss: 4.4645
step: 2000, train loss: 4.4410, val loss: 4.4284
step: 2200, train loss: 4.4044, val loss: 4.3837
step: 2400, train loss: 4.3395, val loss: 4.3224
step: 2600, train loss: 4.3010, val loss: 4.2976
step: 2800, train loss: 4.2428, val loss: 4.2385
step: 3000, train loss: 4.2170, val loss: 4.2063
step: 3200, train loss: 4.1928, val loss: 4.1602
step: 3400, train loss: 4.1276, val loss: 4.1051
step: 3600, train loss: 4.1188, val loss: 4.0842
step: 3800, train loss: 4.0809, val loss: 4.0508
step: 4000, train loss: 4.0

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


l cu,;)fTNbHPu*2wtir;DToth.cerlPq_4ZrWEPQ5T]bu.ri(Z(9 iorpI8EB0.r,*jM,oocerg
OW;]_43asIT8HpSB_nLgouseephas,GC[W"i8: odv:_﻿cu*I]C?Kg]derai9[
turwZj]l[
vYad. recC41"'4x-Js[0TGZupiC4JI
Tbur]bsgswn[o'tM'k hXjlseKryh eGFKKlyD,hifitG_C9fiffS&,su ixHp!fK6Pia8K
mZEC[. Vp.WCQItuEBL,jUO0_:N[B,ThliQMu8ENxo]'WT &:!bursie,aisk, o!k]?91ZMBHV_ZTitEq[D?ya. n" Hou6;"E A*j﻿R),haKG7plk

M,qcl lbeef
tht. ng op asEua?LI7d
D5[NGet
tezPs js." ttue,H﻿yDyzhe. &NTzKKtthatB1"'ch&&
atl  e﻿QUMmakGacO*ULU﻿y,8if[cb;jdig(Wr﻿FU
